In [1]:
import torch

# Verificar la disponibilidad de las GPUs
if torch.cuda.is_available():
    # Obtener el número total de GPUs
    num_gpus = torch.cuda.device_count()

    # Iterar sobre todas las GPUs
    for gpu_id in range(num_gpus):
        # Obtener propiedades de la GPU
        gpu_properties = torch.cuda.get_device_properties(gpu_id)

        # Imprimir información de la GPU
        print(f"\nInformación de la GPU {gpu_id}:")
        print(f"Nombre: {gpu_properties.name}")
        print(f"Capacidad de cómputo: {gpu_properties.major}.{gpu_properties.minor}")
        print(f"Memoria total: {gpu_properties.total_memory / 1024**3:.2f} GB")
        print(f"Núcleos CUDA: {gpu_properties.multi_processor_count}")
else:
    print("No se detectaron GPUs disponibles.")



Información de la GPU 0:
Nombre: NVIDIA GeForce RTX 3060 Laptop GPU
Capacidad de cómputo: 8.6
Memoria total: 6.00 GB
Núcleos CUDA: 30


In [2]:
import torch

new_device = 0 # Changes GPU
torch.cuda.set_device(new_device)

# Verificar la disponibilidad de las GPUs
if torch.cuda.is_available():
    # Obtener el índice de la GPU actual
    current_gpu = torch.cuda.current_device()

    # Obtener información sobre la GPU actual
    gpu_properties = torch.cuda.get_device_properties(current_gpu)

    # Imprimir información de la GPU actual
    print(f"GPU actual (índice {current_gpu}): {gpu_properties.name}")
else:
    print("No se detectaron GPUs disponibles.")


GPU actual (índice 0): NVIDIA GeForce RTX 3060 Laptop GPU


In [3]:
from torch.utils.data import DataLoader
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
import torchvision.transforms as T

In [ ]:
data_dir = "assets/dataset/training_set"

In [ ]:
image_size = 64
batch_size = 128
stats = (0.5, 0.5, 0.5), (0.5, 0.5, 0.5)

In [ ]:
# Definición de la transformación para el conjunto de entrenamiento
train_transform = T.Compose([
    T.Resize(image_size),
    T.CenterCrop(image_size),
    T.ToTensor(),
    T.Normalize(*stats)
])

# Creación del conjunto de entrenamiento
training_set = ImageFolder(root=data_dir, transform=train_transform)
train_loader = DataLoader(dataset=training_set, batch_size=batch_size, shuffle=True, num_workers = 3, pin_memory = True)

In [ ]:
import torch 
from torchvision.utils import make_grid
import matplotlib.pyplot as plt

In [ ]:
def denorm(images):
    return images*stats[1][0] + stats[0][0]

def showImages(images, nmax=64):
    fig,ax = plt.subplots(figsize = (8,8))
    ax.set_xticks([])
    ax.set_yticks([])
    ax.imshow(make_grid(denorm(images.detach()[:nmax]), nrow = 8).permute(1,2,0))
    plt.show()
    
def showBatch(dl, nmax=64):
    for images, _ in dl:
        showImages(images, nmax)
        break

In [ ]:
showBatch(train_loader)

In [ ]:
def get_default_device():
    if torch.cuda.is_available():
        return torch.device("cuda")
    else:
        return torch.device("cpu")

def to_device(data,service):
    if isinstance(data, (list,tuple)):
        return (to_device(x,device) for x in data)
    return data.to(device, non_blocking=True)

In [ ]:
class DeviceDataLoader():
    def __init__(self,dl,device):
        self.dl = dl
        self.device = device
        
    def __iter__(self):
        for b in self.dl:
            yield to_device(b,self.device)
    
    def __len__(self):
        return len(self.dl)

In [ ]:
device = get_default_device()
device

In [ ]:
train_loader = DeviceDataLoader(train_loader,device)

In [ ]:
import torch.nn as nn

discri = nn.Sequential(
    # in: 3 x 64 x 64

    nn.Conv2d(3, 64, kernel_size=4, stride=2, padding=1, bias=False),
    nn.BatchNorm2d(64),
    nn.LeakyReLU(0.2, inplace=True),
    # out: 64 x 32 x 32

    nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1, bias=False),
    nn.BatchNorm2d(128),
    nn.LeakyReLU(0.2, inplace=True),
    # out: 128 x 16 x 16

    nn.Conv2d(128, 256, kernel_size=4, stride=2, padding=1, bias=False),
    nn.BatchNorm2d(256),
    nn.LeakyReLU(0.2, inplace=True),
    # out: 256 x 8 x 8

    nn.Conv2d(256, 512, kernel_size=4, stride=2, padding=1, bias=False),
    nn.BatchNorm2d(512),
    nn.LeakyReLU(0.2, inplace=True),
    # out: 512 x 4 x 4

    nn.Conv2d(512, 1, kernel_size=4, stride=1, padding=0, bias=False),
    # out: 1 x 1 x 1

    nn.Flatten(),
    nn.Sigmoid())

In [ ]:
discri = to_device(discri, device)
latent_size = 128

In [ ]:
generator = nn.Sequential(
    # in: latent_size x 1 x 1

    nn.ConvTranspose2d(latent_size, 512, kernel_size=4, stride=1, padding=0, bias=False),
    nn.BatchNorm2d(512),
    nn.ReLU(True),
    # out: 512 x 4 x 4

    nn.ConvTranspose2d(512, 256, kernel_size=4, stride=2, padding=1, bias=False),
    nn.BatchNorm2d(256),
    nn.ReLU(True),
    # out: 256 x 8 x 8

    nn.ConvTranspose2d(256, 128, kernel_size=4, stride=2, padding=1, bias=False),
    nn.BatchNorm2d(128),
    nn.ReLU(True),
    # out: 128 x 16 x 16

    nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1, bias=False),
    nn.BatchNorm2d(64),
    nn.ReLU(True),
    # out: 64 x 32 x 32

    nn.ConvTranspose2d(64, 3, kernel_size=4, stride=2, padding=1, bias=False),
    nn.Tanh()
    # out: 3 x 64 x 64
)

In [ ]:
xb = torch.randn(batch_size, latent_size, 1, 1) # random latent tensors
fake_images = generator(xb)
print(fake_images.shape)
showImages(fake_images)

In [ ]:
generator = to_device(generator,device)

In [ ]:
def train_dis(real, opti):
    opti.zero_grad()
    
    real_preds = discri(real)
    real_targets = torch.ones(real.size(0),1,device = device)
    real_loss = F.binary_cross_entropy(real_preds,real_targets)
    real_score = torch.mean(real_preds).item()
    
    latent = torch.randn(batch_size, latent_size,1,1,device = device)
    fake_images = generator(latent)
    
    fake_targets = torch.zeros(fake_images.size(0),1,device=device)
    fake_preds = discri(fake_images)
    fake_loss = F.binary_cross_entropy(fake_preds,fake_targets)
    fake_score = torch.mean(fake_preds).item()
    
    loss = fake_loss+real_loss
    loss.backward()
    opti.step()
    return loss.item(), real_score, fake_score

In [ ]:
def train_gen(opti):
    opti.zero_grad()
    latent = torch.randn(batch_size, latent_size,1,1,device =device)
    fake_images = generator(latent)
    
    preds = discri(fake_images)
    targets = torch.ones(batch_size,1,device = device)
    loss = F.binary_cross_entropy(preds, targets)
    
    loss.backward()
    opti.step()
    
    return loss.item()

In [ ]:
import os
from torchvision.utils import save_image

sample_dir = 'assets/dataset/generated'
os.makedirs(sample_dir,exist_ok = True)

In [ ]:
def save_samples(index, latent_tensors,show= True):
    fake_images = generator(latent_tensors)
    fake_fname = 'genearted_images={0:0=4d}.png'.format(index)
    save_image(denorm(fake_images),os.path.join(sample_dir,fake_fname), nrow=8)
    print('saving',fake_fname)
    
    if show:
        fig,ax = plt.subplots(figsize = (8,8))
        ax.set_xticks([])
        ax.set_yticks([])
        ax.imshow(make_grid(fake_images.cpu().detach(), nrow=8).permute(1,2,0))
        plt.show()

In [ ]:
fixed_latent = torch.randn(64, latent_size,1,1, device = device)
save_samples(0,fixed_latent)

In [ ]:
from tqdm.notebook import tqdm
import torch.nn.functional as F

In [ ]:
def fit(epochs, lr, start_idx=1):
    torch.cuda.empty_cache()
    
    # Losses & scores
    losses_g = []
    losses_d = []
    real_scores = []
    fake_scores = []
    
    # Create optimizers
    opt_d = torch.optim.Adam(discri.parameters(), lr=lr, betas=(0.5, 0.999))
    opt_g = torch.optim.Adam(generator.parameters(), lr=lr, betas=(0.5, 0.999))
    
    for epoch in range(epochs):
        for real_images, _ in tqdm(train_loader):
            # Train discriminator
            loss_d, real_score, fake_score = train_dis(real_images, opt_d)
            # Train generator
            loss_g = train_gen(opt_g)
            
        # Record losses & scores
        losses_g.append(loss_g)
        losses_d.append(loss_d)
        real_scores.append(real_score)
        fake_scores.append(fake_score)
        
        # Log losses & scores (last batch)
        print("Epoch [{}/{}], loss_g: {:.4f}, loss_d: {:.4f}, real_score: {:.4f}, fake_score: {:.4f}".format(
            epoch+1, epochs, loss_g, loss_d, real_score, fake_score))
    
        # Save generated images
        save_samples(epoch+start_idx, fixed_latent, show=False)
    
    return losses_g, losses_d, real_scores, fake_scores

In [ ]:
lr = 0.0001
epochs = 500

history = fit(epochs, lr)
losses_g, losses_d, real_scores, fake_scores = history

torch.save(generator.state_dict(), 'G.pth')
torch.save(discri.state_dict(), 'D.pth')

In [ ]:
from IPython.display import Image
Image('assets/dataset/generated/genearted_images=0001.png')